In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\carlo\TrainingsTool\Tennis\Datasets\charting-m-points-from-2017-new.csv", encoding= 'unicode_escape', low_memory=False)

In [3]:
# only take matches where a specific player played
df = df.loc[df["match_id"].str.contains("Novak_Djokovic", case = True)]

In [4]:
# only keep rows where Djoko is serving
df = df[df.Serving == "ND"]

In [5]:
df = df.drop(["isAce", "Gm#", "match_id", "isUnret", "Svr", "Ret", "isRallyWinner", "isForced", "isUnforced", "isDouble", "Serving", "TBpt","Notes", "1stSV", "2ndSV", "1stIn", "2ndIn"], axis=1)

In [6]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [7]:
# seperate the Pts column to create pts_A and pts_B, splitting at the "-" seperator
df[["Pts1", "Pts2"]] = df.Pts.str.split("-", expand = True)

In [8]:
df = df.drop("Pts", axis=1)

In [9]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts1", "Pts2", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [10]:
# Now we need to translate "AD" in Pts1 and Pts2 to "50"
columns = ["Pts1", "Pts2"]
for i in columns:
    df[str(i)] = df[str(i)].str.replace('AD', '50')

In [11]:
df = df.reset_index(drop=True)

In [12]:
test_df = df
test_df = test_df.astype({"1st": str, "2nd": str})

In [13]:
 # join the two ralley colums together into one column with "," seperator
test_df["1st&2nd"] = test_df[["1st", "2nd"]].apply(lambda x: "".join(x), axis=1)

In [14]:
test_df = test_df.drop(["1st", "2nd"], axis = 1)

In [15]:
ralley_df = test_df["1st&2nd"]

In [16]:
# creating a dataframe to store the shot_lists in
shot_list_dataframe = pd.DataFrame()

In [17]:
ralley_df_test = ralley_df.head(20)

In [23]:
possible_shots = ["f", "b", "r", "s", "v", "z", "o", "p", "y", "l", "m", "h", "i", "j", "k", "t", "u"]
serve_direction_second_serve = ["4", "5", "6"]
len_of_longest_ralley = 0
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        # if the char is at position 3 in the ralley and its in serve direction list, its added to shot_list
        if i[char] in serve_direction_second_serve and char == 2:
            shot_list.append(shot)
            shot = ""
            shot = i[char]
             
        elif i[char] not in possible_shots and i[char] not in serve_direction_second_serve:
            shot = shot + str(i[char])
        # in all other cases, shot is done and appended to the shot_list
        else:
            shot_list.append(shot)
            shot = str(i[char])
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list.append(shot)

    # if the last element of the shot list is "nan" we delete that because we dont want to predict "nan"
    if shot_list[-1][-1] == "n" and shot_list[-1][-2] == "a"  and shot_list[-1][-3] == "n" :
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        
    # finding the longest shot_list/ralley in the dataset
    if len(shot_list) > len_of_longest_ralley:
        len_of_longest_ralley = len(shot_list)

    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df = shot_list_dataframe

In [24]:
df = df.drop(0)
df = df.reset_index()
df = df.drop("index", axis = 1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4n,4n,4*,4n,4,4,4n,4d,4,6,6*,6d,4,4,4*,5,4,4,6,4
1,4,6,NaN,5,r28,r37,5,6,b2n#,s1w#,NaN,4,s2n#,r28,NaN,b39,s2n#,f17,r27,r2d#
2,f18,f2n#,NaN,b3n@,f1,b+3,b28,b28,NaN,NaN,NaN,f28,NaN,f1,NaN,b2,NaN,f+3,f+1,NaN
3,f3,NaN,NaN,NaN,r2,m2,f3,f2,NaN,NaN,NaN,f2,NaN,f1,NaN,b2d@,NaN,s3,f3,NaN
4,s2,NaN,NaN,NaN,f3,o2,b2,f1,NaN,NaN,NaN,b1,NaN,f2,NaN,NaN,NaN,z1w#,z1,NaN
5,f1,NaN,NaN,NaN,b3,b2,f3,f1,NaN,NaN,NaN,f1*,NaN,b2,NaN,NaN,NaN,NaN,l3,NaN
6,f3,NaN,NaN,NaN,f2,f3w@,b3,f3,NaN,NaN,NaN,NaN,NaN,f1,NaN,NaN,NaN,NaN,o3*,NaN
7,b3n@,NaN,NaN,NaN,f1,NaN,f2,b3,NaN,NaN,NaN,NaN,NaN,f2,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,f1,NaN,f2,b3,NaN,NaN,NaN,NaN,NaN,f+1,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,f3,NaN,f3,b2,NaN,NaN,NaN,NaN,NaN,f2,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ralley_df_test

0                                    4n4f18f3s2f1f3b3n@
1                                               4n6f2n#
2                                                 4*nan
3                                               4n5b3n@
4                     4r28f1r2f3b3f2f1f1f3b3b3b1r2n#nan
5                                  4r37b+3m2o2b2f3w@nan
6     4n5b28f3b2f3b3f2f2f3s1f1f1f3s3b3b1f2f3b3s3f1f1...
7                              4d6b28f2f1f1f3b3b3b2f1n@
8                                              4b2n#nan
9                                              6s1w#nan
10                                                6*nan
11                                        6d4f28f2b1f1*
12                                             4s2n#nan
13                      4r28f1f1f2b2f1f2f+1f2z3b3i1*nan
14                                                4*nan
15                                        5b39b2b2d@nan
16                                             4s2n#nan
17                                     4f17f+3s3

In [57]:
for i in range(0, len(ralley_df_test)):
    ralley = ralley_df_test[i]
    if ralley[-1] == "n" and ralley[-2] == "a" and ralley[-3] == "n":
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
ralley_df_test

0                                    4n4f18f3s2f1f3b3n@
1                                               4n6f2n#
2                                                    4*
3                                               4n5b3n@
4                        4r28f1r2f3b3f2f1f1f3b3b3b1r2n#
5                                     4r37b+3m2o2b2f3w@
6     4n5b28f3b2f3b3f2f2f3s1f1f1f3s3b3b1f2f3b3s3f1f1...
7                              4d6b28f2f1f1f3b3b3b2f1n@
8                                                 4b2n#
9                                                 6s1w#
10                                                   6*
11                                        6d4f28f2b1f1*
12                                                4s2n#
13                         4r28f1f1f2b2f1f2f+1f2z3b3i1*
14                                                   4*
15                                           5b39b2b2d@
16                                                4s2n#
17                                        4f17f+

In [37]:
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        shot = i[char]
        shot_list.append(shot)
        shot = ""
        
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df_each_char = shot_list_dataframe

In [27]:
df_each_char

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4,4,4,4,4,4,4,4,4,6,6,6,4,4,4,5,4,4,6,4
1,n,n,*,n,r,r,n,d,b,s,*,d,s,r,*,b,s,f,r,r
2,4,6,n,5,2,3,5,6,2,1,n,4,2,2,n,3,2,1,2,2
3,f,f,a,b,8,7,b,b,n,w,a,f,n,8,a,9,n,7,7,d
4,1,2,n,3,f,b,2,2,#,#,n,2,#,f,n,b,#,f,f,#
5,8,n,,n,1,+,8,8,n,n,,8,n,1,,2,n,+,+,n
6,f,#,NaN,@,r,3,f,f,a,a,NaN,f,a,f,NaN,b,a,3,1,a
7,3,,NaN,,2,m,3,2,n,n,NaN,2,n,1,NaN,2,n,s,f,n
8,s,NaN,NaN,NaN,f,2,b,f,,,NaN,b,,f,NaN,d,,3,3,
9,2,NaN,NaN,NaN,3,o,2,1,NaN,NaN,NaN,1,NaN,2,NaN,@,NaN,z,z,NaN
